In [ ]:
import os
import nbformat
from nbconvert import PythonExporter

def convert_ipynb_to_py(folder_path):
    """
    遞迴遍歷指定資料夾，將所有 .ipynb 檔案轉換為 .py 檔案。
    這個版本會自動移除程式碼開頭的 Jupyter 專用魔術指令。
    """
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".ipynb") and not file.startswith("."):
                ipynb_path = os.path.join(root, file)
                py_path = os.path.splitext(ipynb_path)[0] + ".py"

                print(f"Converting: {ipynb_path} -> {py_path}")

                try:
                    # 讀取 .ipynb 檔案
                    with open(ipynb_path, "r", encoding="utf-8") as f:
                        notebook_node = nbformat.read(f, as_version=4)
                    
                    exporter = PythonExporter()
                    # 轉換 .ipynb 到 .py
                    body, _ = exporter.from_notebook_node(notebook_node)

                    # --- 核心修改部分 ---
                    # 透過檢查程式碼行來過濾掉指定的行
                    lines = body.splitlines()
                    filtered_lines = []
                    for line in lines:
                        # 移除 Jupyter 的魔術指令及其轉換後的 Python 函式呼叫
                        if not line.strip().startswith("get_ipython().run_line_magic('load_ext', 'autoreload')") and \
                           not line.strip().startswith("get_ipython().run_line_magic('autoreload', '2')"):
                            filtered_lines.append(line)
                    
                    # 重新組合成單一字串
                    filtered_body = "\n".join(filtered_lines)
                    # --- 核心修改部分結束 ---
                    
                    # 寫入 .py 檔案
                    with open(py_path, "w", encoding="utf-8") as f:
                        f.write(filtered_body)
                
                except Exception as e:
                    print(f"⚠️ Conversion failed for {ipynb_path}: {e}")

    print("✅ All .ipynb files converted.")

if __name__ == "__main__":
    # 更改為你的資料夾名稱
    convert_ipynb_to_py("../application")